In [ ]:
import random

import datasets
import torch
from transformers import pipeline

In [ ]:
df = datasets.load_dataset('kreimben/leetcode_with_youtube_captions')['train']
df = df.to_pandas()
df.sample(3)

In [ ]:
example = df[df['id'] == 1]['cc_content']
example.sample(3)

In [ ]:
N = len(example)
idx = random.randint(0, N)

example_yt_cc = example.values[idx]
example_yt_cc

In [ ]:
# Load the HF pipeline using Gemma 2B from Kaggle
pipe = pipeline(
    "text-generation",
    model='google/gemma-7b-it',
    model_kwargs={"torch_dtype": torch.float16},
    device='cuda',
    max_new_tokens=512
)

In [ ]:
prompt = """
You are a teacher who teaches coding tests to users.
You should explain the algorithm for the coding test problem to the user in detail.
separate the steps but keep it detailed.
Let's think step by step.
"""

In [ ]:
messages = [{
    "role": "user",
    "content": f"{prompt}\n\n{example_yt_cc}"
}]

In [ ]:
# %%time
# 
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(
#     prompt,
#     do_sample=True,
#     temperature=0.1,
#     top_k=5,
#     top_p=0.3,
#     add_special_tokens=True
# )

In [ ]:
# print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
def summarize_example(text):
    global prompt
    messages = [{
        "role": "user",
        "content": f"{prompt}\n\n{text}"
    }]
    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.1,
        top_k=5,
        top_p=0.3,
        add_special_tokens=True
    )

    return outputs[0]["generated_text"][len(prompt):]

In [ ]:
df['summary'] = df['summary'].apply(summarize_example)